In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

df = pd.read_csv("./insurance.csv")

# EDA

## Phase 1: The First Look

In [ ]:
print(df.head())
print(df.shape)
print(df.info())

# Do the column names make sense? Does the data in the rows look correct?

In [ ]:
# What are the min/max values (for spotting outliers)? Are the mean and median (50%) far apart (which indicates skew)?

print(df.describe())

In [ ]:
print(f"Total duplicated rows: {df.duplicated().sum()}")

# df = df.drop_duplicates()

## Phase 2: Univariate Analysis (One Variable at a Time)

In [ ]:
# What is the shape of this data? Is it symmetric (a "normal" bell curve)? Is it skewed? Does it have one peak (unimodal) or multiple peaks (multimodal)?
sns.histplot(df['expenses'], kde=True, bins=30)
plt.title('Distribution of Medical Charges')
plt.show()

In [ ]:
# Are there data points far outside the "whiskers"? These are outliers.
sns.boxplot(x=df['bmi'])
plt.title('Box Plot of BMI')
plt.show()

In [ ]:
# Are the classes balanced (e.g., 50/50 smokers/non-smokers) or imbalanced (e.g., 90/10)?

print(df['smoker'].value_counts())

print(df['smoker'].value_counts(normalize=True))

In [ ]:
sns.countplot(x=df['region'])
plt.title('Count of Patients by Region')
plt.show()

## Phase 3: Bivariate Analysis (Two Variables at a Time)

In [ ]:
# Is there a relationship? Is it linear (a straight line)? Is it positive (goes up) or negative (goes down)?
sns.scatterplot(x=df['age'], y=df['expenses'])
plt.title('Age vs. Medical expenses')
plt.show()

In [ ]:
print(df['age'].corr(df['expenses']))

In [ ]:
# Does the distribution of the numerical variable change for each category?
sns.boxplot(x=df['smoker'], y=df['expenses'])
plt.title('Smoker Status vs. Medical expenses')
plt.show()

In [ ]:
sns.countplot(x=df['region'], hue=df['smoker'])
plt.title('Smoker Distribution by Region')
plt.show()

## Phase 4: Multivariate Analysis (3+ Variables)

In [ ]:
# Which features are most correlated with the target (expenses)? Are any of your features highly correlated with each other (this is multicollinearity and can be a problem)?

numerical_df = df.select_dtypes(include=['float64', 'int64'])

corr_matrix = numerical_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
# Does the relationship between age and expenses depend on whether the person is a smoker? (This is an interaction effect).

sns.scatterplot(x=df['age'], y=df['expenses'], hue=df['smoker'])
plt.title('Age vs. expenses (Colored by Smoker)')
plt.show()

## Phase 5: Summarize Findings & Plan Action

In [ ]:
"""
Goal: Turn your insights into a concrete plan for preprocessing and feature engineering.

Example Findings Summary:

    Target: expenses is heavily right-skewed.

    Missing Data: The dataset is clean, no missing values to impute.

    Categorical: sex, smoker, and region all need to be one-hot encoded.

    Key Predictors: smoker is the strongest predictor by far. age and bmi are also positively correlated with expenses.

    Interactions: The effect of age on expenses is much stronger for smokers.

Resulting Action Plan:

    Preprocessing: Log-transform expenses (i.e., df['log_expenses'] = np.log(df['expenses'])) and use this new column as your target y.

    Preprocessing: One-hot encode sex, smoker, and region.

    Feature Engineering: Create an interaction term, such as age * smoker_status or bmi * smoker_status, 
    as these relationships are clearly not simple.

    Model Selection: Since the smoker effect is so strong, a simple LinearRegression will likely do well, 
    but a DecisionTree or RandomForest will be able to capture the non-linear interactions automatically."""

# Preprocessing

In [ ]:
"""
Dropping: Remove the entire row or column if it has missing data. This is okay if you have a huge dataset and only a few missing rows, 
but it's generally wasteful.

Imputation: Fill in the missing values. This is the preferred method.

    Numerical: Fill with the mean or median (median is better if the data is skewed).

    Categorical: Fill with the mode (the most frequent category).
"""
print(df.isnull().sum())

from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='median')
df['bmi'] = num_imputer.fit_transform(df[['bmi']])

cat_imputer = SimpleImputer(strategy='most_frequent')
df[['region']] = cat_imputer.fit_transform(df[['region']])

In [ ]:
"""
We use One-Hot Encoding. This technique creates new binary (0 or 1) columns for each category.

sex: "male", "female" → Becomes sex_male (1 if male, 0 if female).

smoker: "yes", "no" → Becomes smoker_yes (1 if smoker, 0 if not).

After running this, your sex column is gone, replaced by sex_male. Your smoker column is gone, replaced by smoker_yes. 
Your region column is gone, replaced by region_northwest, 
region_southeast, and region_southwest. (The fourth one, northeast, is represented when all the others are 0).
"""
df_processed = df.copy()

df_processed = pd.get_dummies(df_processed, 
                              columns=['sex', 'smoker', 'region'], 
                              drop_first=True)

print(df_processed.head())

In [ ]:

df_processed['log_expenses'] = np.log(df_processed['expenses'])

df_processed = df_processed.drop('expenses', axis=1)

sns.histplot(df_processed['log_expenses'], kde=True, bins=30)
plt.title('Distribution of Log-Transformed Medical expenses')
plt.show()

# When you make predictions later, they will be in "log-dollars," 
# so you must transform them back using np.exp() to get the actual dollar amount.

In [ ]:
y = df_processed['log_expenses']

X = df_processed.drop('log_expenses', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,     
    random_state=42
)

print(f"Training set shape (X): {X_train.shape}")
print(f"Testing set shape (X): {X_test.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

columns_to_scale = ['age', 'bmi', 'children']

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print(X_train_scaled.describe())

# Modeling

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("✅ Model trained successfully!")

In [ ]:

print("\n--- Model Parameters ---")
print(f"Intercept (b): {model.intercept_:.4f}")

coefficients = pd.DataFrame(
    data=model.coef_, 
    index=X.columns, 
    columns=['Coefficient']
)

coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()
print("\nModel Coefficients (w):")
print(coefficients.sort_values(by='Abs_Coefficient', ascending=False))

In [ ]:
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

print("\n✅ Predictions made on train and test data.")

In [ ]:
print("\n--- Model Evaluation ---")
print("Test Set Performance:")

r2_test = r2_score(y_test, y_pred_test)
print(f"  R-squared (R²): {r2_test:.4f}")

y_test_orig = np.exp(y_test)
y_pred_test_orig = np.exp(y_pred_test)

mae_test = mean_absolute_error(y_test_orig, y_pred_test_orig)
print(f"  Mean Absolute Error (MAE): ${mae_test:,.2f}")

rmse_test = np.sqrt(mean_squared_error(y_test_orig, y_pred_test_orig))
print(f"  Root Mean Squared Error (RMSE): ${rmse_test:,.2f}")

print("\nTrain Set Performance (for comparison):")

r2_train = r2_score(y_train, y_pred_train)
print(f"  R-squared (R²): {r2_train:.4f}")

y_train_orig = np.exp(y_train)
y_pred_train_orig = np.exp(y_pred_train)
mae_train = mean_absolute_error(y_train_orig, y_pred_train_orig)
print(f"  Mean Absolute Error (MAE): ${mae_train:,.2f}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_orig, y=y_pred_test_orig, alpha=0.7)

min_val = min(y_test_orig.min(), y_pred_test_orig.min())
max_val = max(y_test_orig.max(), y_pred_test_orig.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)

plt.xlabel("Actual expenses ($)")
plt.ylabel("Predicted expenses ($)")
plt.title("Actual vs. Predicted Medical expenses (Test Set)")
plt.show()

# Improving The Model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso

def evaluate_model(model_name, model, X_test, y_test):
    """
    Makes predictions and prints evaluation metrics (R², MAE, RMSE).
    Handles the inverse transform (np.exp) for error metrics.
    """
    print(f"--- Evaluating Model: {model_name} ---")
    
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    print(f"  R-squared (R²): {r2:.4f}")
    
    y_test_orig = np.exp(y_test)
    y_pred_orig = np.exp(y_pred)
    
    mae = mean_absolute_error(y_test_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
    
    print(f"  Mean Absolute Error (MAE): ${mae:,.2f}")
    print(f"  Root Mean Squared Error (RMSE): ${rmse:,.2f}\n")
    
    return r2, mae, rmse

In [ ]:
model_performance = {}

baseline_pipe = Pipeline([
    ('model', LinearRegression())
])
baseline_pipe.fit(X_train, y_train)
r2, mae, rmse = evaluate_model("Baseline Linear Regression", baseline_pipe, X_test, y_test)
model_performance['Baseline_Linear'] = (r2, mae)

In [ ]:
print("Building Ridge (L2) Regularization Model...")

ridge_pipe = Pipeline([
    ('model', Ridge(alpha=1.0))  # alpha=1.0 is a common default to start with. A higher alpha = a stronger penalty / simpler model.
])

ridge_pipe.fit(X_train, y_train)

r2, mae, rmse = evaluate_model("Ridge Regression (L2)", ridge_pipe, X_test, y_test)
model_performance['Ridge_L2'] = (r2, mae)

In [ ]:
print("Building Lasso (L1) Regularization Model...")

lasso_pipe = Pipeline([
    ('model', Lasso(alpha=0.01))
])

lasso_pipe.fit(X_train, y_train)

r2, mae, rmse = evaluate_model("Lasso Regression (L1)", lasso_pipe, X_test, y_test)
model_performance['Lasso_L1'] = (r2, mae)

lasso_coefs = lasso_pipe.named_steps['model'].coef_

coef_df = pd.DataFrame(
    data=lasso_coefs, 
    index=X_train.columns, 
    columns=['Coefficient']
)

print("Lasso Coefficients:")
print(coef_df.sort_values(by='Coefficient', ascending=False))

In [ ]:
print("Building Polynomial Regression Model...")

poly_pipe = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)), # 'degree=2' is standard. degree=3 or higher will likely overfit.
    ('model', LinearRegression())
])

poly_pipe.fit(X_train, y_train)

r2, mae, rmse = evaluate_model("Polynomial Regression (Degree 2)", poly_pipe, X_test, y_test)
model_performance['Polynomial_D2'] = (r2, mae)

In [ ]:
print("Building Polynomial + Ridge Regression Model...")

poly_ridge_pipe = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', Ridge(alpha=10.0)) # We use a stronger alpha to control the many new features
])

poly_ridge_pipe.fit(X_train, y_train)

r2, mae, rmse = evaluate_model("Polynomial (D2) + Ridge (alpha=10)", poly_ridge_pipe, X_test, y_test)
model_performance['Poly_Ridge'] = (r2, mae)

In [ ]:
print("--- Final Model Comparison ---")

results_df = pd.DataFrame.from_dict(
    model_performance, 
    orient='index', 
    columns=['R_squared', 'Mean_Absolute_Error']
)

print(results_df.sort_values(by='R_squared', ascending=False))